In [1]:
import os
import config
v1, v2, v3 = config.spark_env("msi")
os.environ['SPARK_VERSION'] = v1
os.environ['JAVA_HOME'] = v2
os.environ['SPARK_HOME'] = v3
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import array, col, explode, lit, struct
from pyspark.sql import DataFrame
from typing import Iterable
import numpy as np

In [2]:
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [ ]:
def sql_to_spark(table):
    from pyspark.sql import Row
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    query_list = []
    for i in query:
        q_len = len(i)
        temp_dict = {}
        for j in range(q_len):
            key = steve[j]
            value = i[j]
            if value == None:
                value = float(0)
            temp_dict[key] = value
            if j == (q_len - 1):
                query_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in query_list)
    return df

In [3]:
def sql_to_spark(table):
    from pyspark.sql import Row
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    query_list = []
    for i in query:
        q_len = len(i)
        temp_dict = {}
        for j in range(q_len):
            key = steve[j]
            value = i[j]
            if value == None:
                value = float(0)
            temp_dict[key] = value
            if j == (q_len - 1):
                query_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in query_list)
    return df

def query_maker(table):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    return (query, steve)

def spark_maker(query,country):
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list) 
    return df

def sql_spark_trade(table, country):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).filter(table_var.c.country_or_area == country).all()
    steve = table_var.metadata.tables[table].columns.keys()
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list)
    df = df.withColumn("year",df.year.cast('int'))
    df = df.withColumnRenamed("country_or_area","country")
    df_export = df.filter(df['flow'] == "Export")
    df_import = df.filter(df['flow'] == "Import")
    return (df_export, df_import)

def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)

def min_max(df,column):
    max_ = df.agg({column: 'max'})
    min_ = df.agg({column: 'min'})
    return (min_, max_)

def min_int(df,column):
    arr = np.array(df.select(column).collect())
    y_l = []
    for i in arr:
        y_l.append(int(i[0]))
    min_ = min(y_l)
    max_ = max(y_l)
    return (min_, max_)

In [4]:
from sqlalchemy import MetaData, Table
metadata = MetaData(engine)
global_trade = Table("global_trade", metadata, autoload=True, autoload_with=engine)
trade_schema = global_trade.metadata.tables["global_trade"].columns.keys()

In [5]:
# spark_india=sql_to_spark("india_export_data")
spark_emissions=sql_to_spark('global_emissions')
spark_emissions = spark_emissions.withColumnRenamed("Entity","country")
spark_emissions = spark_emissions.withColumnRenamed("Year","year")
spark_gdp=sql_to_spark('gdp_data')
gdp_col = spark_gdp.columns
gdp_col2 = gdp_col[2:]
gdp_static_cols = gdp_col[:2]
spark_gdp_2 = melt(spark_gdp, id_vars=gdp_static_cols, value_vars=gdp_col2)
spark_gdp_2 = spark_gdp_2.withColumn("variable",spark_gdp_2.variable.cast('int'))
spark_gdp_2 = spark_gdp_2.withColumnRenamed("variable","year")
spark_gdp_2 = spark_gdp_2.withColumnRenamed("value","GDP")
spark_gdp_2 = spark_gdp_2.withColumnRenamed("Country ","country")
spark_gdp_2 = spark_gdp_2.select(['country','year','GDP'])
india_export_df, india_import_df = sql_spark_trade("global_trade","India")
india_export_df_min, india_export_df_max = min_int(india_export_df, 'year')
india_import_df_min, india_import_df_max = min_int(india_import_df, 'year')
india_gdp_spark = spark_gdp_2.filter(spark_gdp_2["country"] == "India")
india_gdp_spark_min, india_gdp_spark_max = min_int(india_gdp_spark, "year")
india_emissions = spark_emissions.filter(spark_emissions["country"] == "India")
india_emissions_min, india_emissions_max = min_int(india_emissions, 'year')
years_df = pd.DataFrame([{"min":india_export_df_min, "max":india_export_df_max},
 {"min":india_import_df_min, "max":india_import_df_max},
 {"min":india_gdp_spark_min, "max":india_gdp_spark_max},
 {"min":india_emissions_min, "max":india_emissions_max},
 {"min":spark_india_min,"max":spark_india_max}])
lower_bound = years_df['min'].max()
upper_bound = years_df['max'].min()
years_df = pd.DataFrame([{"min":india_export_df_min, "max":india_export_df_max},
 {"min":india_import_df_min, "max":india_import_df_max},
 {"min":india_gdp_spark_min, "max":india_gdp_spark_max},
 {"min":india_emissions_min, "max":india_emissions_max},
#  {"min":spark_india_min,"max":spark_india_max}])
                        ])
lower_bound = years_df['min'].max()
upper_bound = years_df['max'].min()
india_gdp_filt = india_gdp_spark.filter(india_gdp_spark["year"] >= lower_bound)
india_gdp_filt = india_gdp_filt.filter(india_gdp_filt["year"] <= upper_bound)
india_emissions_filt = india_emissions.filter(india_emissions["year"] >= lower_bound)
india_emissions_filt = india_emissions_filt.filter(india_emissions_filt["year"] <= upper_bound)
india_import_filt = india_import_df.filter(india_import_df["year"] >= lower_bound)
india_import_filt = india_import_filt.filter(india_import_filt["year"] <= upper_bound)
india_export_filt = india_export_df.filter(india_export_df["year"] >= lower_bound)
india_export_filt = india_export_filt.filter(india_export_filt["year"] <= upper_bound)
merged_df = india_gdp_filt.join(india_emissions_filt, india_gdp_filt.year == india_emissions_filt.year, 'outer') \
.select(india_gdp_filt.country ,india_gdp_filt.year,india_gdp_filt.GDP, india_emissions_filt.annual_co2_emissions_tonnes) \
.distinct()
merged_df = merged_df.orderBy(merged_df.year.asc())
india_import_sum = india_import_filt.groupBy('year').sum()
india_import_sum = india_import_sum.orderBy(india_import_sum.year.asc())
india_import_sum = india_import_sum.withColumnRenamed("sum(trade_usd)","import_trade_sum_usd")
india_import_sum = india_import_sum.withColumnRenamed("sum(weight_kg)","import_weight_sum_kg")
india_import_sum = india_import_sum.withColumnRenamed("sum(quantity)","import_quantity_sum")
india_export_sum = india_export_filt.groupBy('year').sum()
india_export_sum = india_export_sum.orderBy(india_export_sum.year.asc())
india_export_sum = india_export_sum.withColumnRenamed("sum(trade_usd)","export_trade_sum_usd")
india_export_sum = india_export_sum.withColumnRenamed("sum(weight_kg)","export_weight_sum_kg")
india_export_sum = india_export_sum.withColumnRenamed("sum(quantity)","export_quantity_sum")
merged_df2 = merged_df.join(india_import_sum, merged_df.year == india_import_sum.year, 'outer') \
.select(merged_df.country ,merged_df.year,merged_df.GDP, merged_df.annual_co2_emissions_tonnes,
       india_import_sum['import_trade_sum_usd'],india_import_sum['import_weight_sum_kg'],india_import_sum['import_quantity_sum']) \
.distinct()
merged_df2 = merged_df2.orderBy(merged_df2.year.asc())
final_merged_spark = merged_df2.join(india_export_sum, merged_df2.year == india_export_sum.year, 'outer') \
.select(merged_df2.country ,merged_df2.year,merged_df2.GDP, merged_df.annual_co2_emissions_tonnes,
       merged_df2['import_trade_sum_usd'],merged_df2['import_weight_sum_kg'],merged_df2['import_quantity_sum'],
       india_export_sum['export_trade_sum_usd'],india_export_sum['export_weight_sum_kg'],india_export_sum['export_quantity_sum']
       ) \
.distinct()
final_merged_spark = final_merged_spark.orderBy(final_merged_spark.year.asc())

In [6]:
spark_emissions=sql_to_spark('global_emissions')
spark_emissions = spark_emissions.withColumnRenamed("Entity","country")
spark_emissions = spark_emissions.withColumnRenamed("Year","year")

In [8]:
spark_gdp=sql_to_spark('gdp_data')
gdp_col = spark_gdp.columns
gdp_col2 = gdp_col[2:]
gdp_static_cols = gdp_col[:2]
spark_gdp_2 = melt(spark_gdp, id_vars=gdp_static_cols, value_vars=gdp_col2)
spark_gdp_2 = spark_gdp_2.withColumn("variable",spark_gdp_2.variable.cast('int'))
spark_gdp_2 = spark_gdp_2.withColumnRenamed("variable","year")
spark_gdp_2 = spark_gdp_2.withColumnRenamed("value","GDP")
spark_gdp_2 = spark_gdp_2.withColumnRenamed("Country ","country")
spark_gdp_2 = spark_gdp_2.select(['country','year','GDP'])

In [9]:
india_export_df, india_import_df = sql_spark_trade("global_trade","India")
india_export_df_min, india_export_df_max = min_int(india_export_df, 'year')
india_import_df_min, india_import_df_max = min_int(india_import_df, 'year')

In [11]:
india_gdp_spark = spark_gdp_2.filter(spark_gdp_2["country"] == "India")
india_gdp_spark_min, india_gdp_spark_max = min_int(india_gdp_spark, "year")

In [12]:
india_emissions = spark_emissions.filter(spark_emissions["country"] == "India")
india_emissions_min, india_emissions_max = min_int(india_emissions, 'year')

In [13]:
# spark_india_min, spark_india_max = min_int(spark_india, 'year')

In [77]:
years_df = pd.DataFrame([{"min":india_export_df_min, "max":india_export_df_max},
 {"min":india_import_df_min, "max":india_import_df_max},
 {"min":india_gdp_spark_min, "max":india_gdp_spark_max},
 {"min":india_emissions_min, "max":india_emissions_max},
#  {"min":spark_india_min,"max":spark_india_max}])
                        ])
lower_bound = years_df['min'].max()
upper_bound = years_df['max'].min()

In [78]:
india_gdp_filt = india_gdp_spark.filter(india_gdp_spark["year"] >= lower_bound)
india_gdp_filt = india_gdp_filt.filter(india_gdp_filt["year"] <= upper_bound)
# india_gdp_filt = india_gdp_filt.orderBy(india_gdp_filt.year.asc())

In [79]:
india_emissions_filt = india_emissions.filter(india_emissions["year"] >= lower_bound)
india_emissions_filt = india_emissions_filt.filter(india_emissions_filt["year"] <= upper_bound)
# india_emissions_filt = india_emissions_filt.orderBy(india_emissions_filt.year.asc())

In [80]:
india_import_filt = india_import_df.filter(india_import_df["year"] >= lower_bound)
india_import_filt = india_import_filt.filter(india_import_filt["year"] <= upper_bound)
# india_import_filt = india_import_filt.orderBy(india_import_filt.year.asc())

In [81]:
india_export_filt = india_export_df.filter(india_export_df["year"] >= lower_bound)
india_export_filt = india_export_filt.filter(india_export_filt["year"] <= upper_bound)
# india_export_filt = india_export_filt.orderBy(india_export_filt.year.asc())

In [82]:
merged_df = india_gdp_filt.join(india_emissions_filt, india_gdp_filt.year == india_emissions_filt.year, 'outer') \
.select(india_gdp_filt.country ,india_gdp_filt.year,india_gdp_filt.GDP, india_emissions_filt.annual_co2_emissions_tonnes) \
.distinct()

merged_df = merged_df.orderBy(merged_df.year.asc())

In [30]:
# india_import_sum = india_import_filt.groupBy('year').sum().collect()

In [83]:
india_import_sum = india_import_filt.groupBy('year').sum()
india_import_sum = india_import_sum.orderBy(india_import_sum.year.asc())
india_import_sum = india_import_sum.withColumnRenamed("sum(trade_usd)","import_trade_sum_usd")
india_import_sum = india_import_sum.withColumnRenamed("sum(weight_kg)","import_weight_sum_kg")
india_import_sum = india_import_sum.withColumnRenamed("sum(quantity)","import_quantity_sum")

In [84]:
india_export_sum = india_export_filt.groupBy('year').sum()
india_export_sum = india_export_sum.orderBy(india_export_sum.year.asc())
india_export_sum = india_export_sum.withColumnRenamed("sum(trade_usd)","export_trade_sum_usd")
india_export_sum = india_export_sum.withColumnRenamed("sum(weight_kg)","export_weight_sum_kg")
india_export_sum = india_export_sum.withColumnRenamed("sum(quantity)","export_quantity_sum")

In [85]:
merged_df2 = merged_df.join(india_import_sum, merged_df.year == india_import_sum.year, 'outer') \
.select(merged_df.country ,merged_df.year,merged_df.GDP, merged_df.annual_co2_emissions_tonnes,
       india_import_sum['import_trade_sum_usd'],india_import_sum['import_weight_sum_kg'],india_import_sum['import_quantity_sum']) \
.distinct()
merged_df2 = merged_df2.orderBy(merged_df2.year.asc())

In [87]:
final_merged_spark = merged_df2.join(india_export_sum, merged_df2.year == india_export_sum.year, 'outer') \
.select(merged_df2.country ,merged_df2.year,merged_df2.GDP, merged_df.annual_co2_emissions_tonnes,
       merged_df2['import_trade_sum_usd'],merged_df2['import_weight_sum_kg'],merged_df2['import_quantity_sum'],
       india_export_sum['export_trade_sum_usd'],india_export_sum['export_weight_sum_kg'],india_export_sum['export_quantity_sum']
       ) \
.distinct()
final_merged_spark = final_merged_spark.orderBy(final_merged_spark.year.asc())

In [91]:
final_merged_spark.show(truncate=False)

+-------+----+-----------+---------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
|country|year|GDP        |annual_co2_emissions_tonnes|import_trade_sum_usd|import_weight_sum_kg|import_quantity_sum|export_trade_sum_usd|export_weight_sum_kg|export_quantity_sum|
+-------+----+-----------+---------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
|India  |1990|1236.671208|6.165790984E8              |37086019677         |3.2819250453E10     |3.6282271392E10    |27688482159         |3.8376259571E10     |4.480699398E10     |
|India  |1991|1265.917481|6.567408929E8              |31062661051         |3.4226080286E10     |3.8234796507E10    |27430173100         |3.6816498359E10     |3.9646091225E10    |
|India  |1992|1338.664595|6.987569497E8              |39084634969         |4.2689104735E10     |4.7599677